<a href="https://colab.research.google.com/github/linshaochieh2019/test/blob/main/ventilator_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount to Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [170]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import time
from datetime import datetime

import torch
from torch.utils.data import random_split, DataLoader, Dataset

In [171]:
train_csv = '/content/drive/MyDrive/Data/ventilator1/train.csv'

train = pd.read_csv(train_csv)

print('# data entries: {}'.format(len(train)))
print('# breath id: {}'.format(len(train['breath_id'].unique())))

# data entries: 6036000
# breath id: 75450


In [172]:
def evenly_spaced(df, breath_id):
  
  df_temp = df[df['breath_id'] == breath_id]
  df_temp = df_temp[['time_step', 'pressure', 'u_in']]
  df_temp['mask'] = np.zeros(len(df_temp))

  data_temp = {'time_step': np.arange(0,2.5,0.025),
               'pressure': np.nan,
               'u_in': np.nan,
               'mask':np.ones(len(np.arange(0,2.5,0.025)))}
  
  df_gen = pd.DataFrame.from_dict(data_temp)
  df_new = pd.concat((df_gen, df_temp))
  df_new = df_new.sort_values(by='time_step', ascending=True)
  df_new = df_new.interpolate(method='linear', axis=0)
  df_new = df_new.iloc[1:]
  df_new = df_new[df_new['mask'] == 1]
  df_new = df_new.iloc[:, :3]

  temp_dict = df_new.to_dict(orient='list')
  temp_dict['breath_id'] = np.ones(len(df_new)) * breath_id

  return temp_dict

In [168]:
data_dict = {
              'breath_id': [],
              'pressure': [],
              'u_in': [],
              'time_step': [],
             }

ls_cols = ['breath_id', 'pressure', 'u_in', 'time_step']

ls_breath_id = train['breath_id'].unique()
for breath_id in tqdm(ls_breath_id):
  data_temp = evenly_spaced(df=train, breath_id=breath_id)
  for col in ls_cols:
    data_dict[col].extend(data_temp[col])

  3%|▎         | 2515/75450 [00:28<13:29, 90.14it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
 36%|███▋      | 27471/75450 [05:13<09:04, 88.19it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
 60%|█████▉    | 45109/75450 [08:32<05:44, 88.18it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy

14.194433987140656


In [189]:
# generate rc_map
ls_rc_maps = []

for breath_id in tqdm(ls_breath_id):
  data_temp = train[train['breath_id'] == breath_id].iloc[0][['breath_id', 'R', 'C']].to_dict()
  ls_rc_maps.append(data_temp)

100%|██████████| 75450/75450 [10:01<00:00, 125.34it/s]


In [192]:
df_temp = pd.DataFrame(ls_rc_maps)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_temp['R'] = le.fit_transform(df_temp['R'])
df_temp['C'] = le.fit_transform(df_temp['C'])

df_temp.to_csv('/content/drive/MyDrive/Data/ventilator1/rcmap.csv')

# Discussion



1.   Interpolation among time steps - to standardize the unevenly time gaps.
2.   'R', 'C' columns get standardized
3.  

